In [21]:
import pandas as pd
import numpy as np
import math
import datetime

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur1d = pd.read_csv("../Dati/turni_f/furgoncino1_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur2d = pd.read_csv("../Dati/turni_f/furgoncino2_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

def getName(nome):
    city = dft[get_row(dft, nome).values].index[0]
    return city

In [22]:
def new_route(schedule, dframe):
    times = []
    keys = []
    #row = dft[get_row(dft, 'Cremona, sesto 39').values]
    row = dframe[get_row(dframe, schedule[0]).values]
    key1 = ''
    key2 = ''
    
    for i in range(1,len(schedule)):
        key1 = row.index[0]
        paese=schedule[i]
        if 'Martignana' in paese:
            paese = 'Martignana di Po'
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dframe, paese).values].values[0][0]
        times.append(round(value))
        row = dframe[get_row(dframe, paese).values]
        try:
            key2 = row.index[0]
        except:
            raise Exception("ciao")
        keys.append(key1 + ' - ' + key2)
    
    steps_times1 = pd.Series(times , keys)
    times1 = times[:]
    return steps_times1

In [23]:
new_route(fur1["lunedi"].dropna(), dft)

/home/alberto/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


Series([], dtype: float64)

In [3]:
#returns a pd.Series: keys = city1 - city2, values = times from city1 to city2
def schedule_time(new_schedule, dft):
    times = []
    #row = dft[get_row(dft, 'Cremona, sesto 39').values]
    row = dft[get_row(dft, new_schedule[0]).values]
    keys = []
    key1 = ''
    key2 = ''
    for i in range(1,len(new_schedule)):
        key1 = row.index[0]
        paese=new_schedule[i]

        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    steps_times = pd.Series(times , keys)
    return steps_times

In [4]:
def timeDistance(start, end, dft):
    start_city_row = dft[get_row(dft, start).values]
    return start_city_row.loc[:,get_column(dft, end).values].values[0][0]

In [5]:
#Data una lista ottenuta da route restituisce la lista dei nomi delle città
def cities_list(serie):
    cities = []
    keys = serie.keys()
    for i in range(0, len(serie)):
        cities = cities + [keys[i].split('-')[0].strip()]
    if len(keys[i].split('-')) > 2: 
        return cities + [keys[i].split('-')[len(keys[i].split('-'))-1].strip()]
    else:
        return cities + [keys[i].split('-')[1].strip()]

In [6]:
#first attempt to local search algorithm
def find_best_position(steps_times, to_insert, dframe):
    times = []
    key2 = []
    for i in range(len(steps_times)):
        cities = steps_times.keys()[i].split('-')
        if(cities[0].strip()=="San Bassano, Mons. Frosi (ex piazza Scuole)"):
            row1 = dframe[get_row(dframe, 'San Bassano, Mons. Frosi').values]
        else:
            row1 = dframe[get_row(dframe, cities[0].strip()).values]
        #time from city 1 to current (to insert)
        #print("From: " + cities[0].strip() + " to: " + to_insert)
        f_c1_to_curr = row1.loc[:,get_column(dframe, to_insert).values].values[0][0]
        #print("Done\n")
        row2 = dframe[get_row(dframe, to_insert).values]
        #time from current (to insert) to city 2
        f_curr_to_c2 = row2.loc[:,get_column(dframe, cities[1].strip()).values].values[0][0]
        times.append(f_c1_to_curr + f_curr_to_c2)
    s3 = pd.Series(times, steps_times.keys())
    return s3

In [7]:
def best_index(lista, schedule, to_insert, dframe):
    idx = find_best_position(schedule, to_insert, dframe).idxmin()
    a = idx.split("-")[0].strip()
    new_lista = [obj.lower() for obj in lista]
    try:
        return new_lista.index(a.lower())+1
    except:
        #problema nome diverso
        if(a.lower() == 'casale cremasco'):
            a = 'Casale Cremasco-Vidolasco, Roma, 56'
            return lista.index(a)+1
        elif(a.lower() == 'gadesco'):
            a = 'Gadesco-Pieve Delmona, Giovanni Lonati 9'
            return lista.index(a)+1
        elif(a=="San Bassano, Mons. Frosi (ex piazza Scuole)"):
            a = "San Bassano, Mons. Frosi"
            return lista.index(a)+1

In [16]:
def definitive_local_search(fur, n, dft):
    #Schedulazione iniziale, e sequenza dei nomi iniziali, una per ciclare e una copia per effettuare pop e insert
    pre_schedule = new_route(fur, dft)
    lista = [getName(x) for x in fur]
    sequence = lista[:]
    #Costo iniziale del percorso greedy
    if(n==1):
         pre_cost = sum(pre_schedule.values) + timeDistance('Cremona, Sesto 39', lista[0], dft) + timeDistance(lista[-1], 'Ostiano, Mazzini, 1', dft)
    else:
         pre_cost = sum(pre_schedule.values) + timeDistance('Cremona, Sesto 39', lista[0], dft) + timeDistance(lista[-1], 'San Giovanni in Croce, Feudatari, 25 A', dft)
    new_cost = 0
    local_best_cost = math.inf
    local_best_schedule = []

    #Eseguo finchè ho dei miglioramenti e quindi i nuovi costi sono migliori del migliore al passo prima
    while(True):
        #Prendo ogni città in lista, la inserisco nella posizione migliore, e tra tutte le città prendo quella che da lo
        #spostamento più vantaggioso.
        #Se questo è anche migliore del risultato precedente utilizzo la nuova sequenza come passo del ciclo successivo
        for i in range(len(lista)):
            #Copia della lista su cui lavorare
            sequence = lista[:]
            to_insert = lista[i]
            sequence.remove(to_insert)
            curr_schedule = new_route(sequence, dft)
            index = best_index(sequence, curr_schedule, to_insert, dft)
            try:
                sequence.insert(index, to_insert)
            except:
                print(sequence)
                print(to_insert)
                raise Exception("Errororor")
            curr_schedule = new_route(sequence, dft)
            if(n==1):
                new_cost = sum(curr_schedule.values) + timeDistance('Cremona, Sesto 39', sequence[0], dft) + timeDistance(sequence[-1], 'Ostiano, Mazzini, 1', dft)
            else:
                new_cost = sum(curr_schedule.values) + timeDistance('Cremona, Sesto 39', sequence[0], dft) + timeDistance(sequence[-1], 'San Giovanni in Croce, Feudatari, 25 A', dft)
            if(new_cost<local_best_cost):
                local_best_cost = new_cost
                local_best_schedule = curr_schedule[:]
        if(local_best_cost<pre_cost):
            pre_cost = local_best_cost
            pre_schedule = local_best_schedule[:]
            lista = cities_list(pre_schedule)
        else:
            break
    return cities_list(pre_schedule), pre_cost

In [9]:
#TIME LOCAL SEARCH
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur1[day].dropna(), 1, dft)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('Ostiano, Mazzini, 1')
    to_write[day] = pd.Series(sch)

In [10]:
to_write.to_csv("../Dati/turni_f/ls1_shifts.csv", index=False, sep=";")

In [9]:
week= ['martedi', 'mercoledi', 'giovedi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur2[day].dropna(), 2, dft)
    print(day)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('San Giovanni in Croce, Feudatari, 25 A')
    to_write[day] = pd.Series(sch)
to_write.to_csv("../Dati/turni_f/ls2_shifts.csv", index=False, sep=";")
to_write

martedi
mercoledi
giovedi


,martedi,mercoledi,giovedi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Cremona, Felice Cavallotti, 2","Cremona, Palosca n. 2","Cremona, Ugolani Dati, 4"
2,"Bonemerse, Roma, 25","Casalbuttano ed Uniti, Jacini, 23","Pizzighettone, Giuseppe Garibaldi, 18"
3,"Sospiro, Tosi","Soresina, Giacomo Matteotti, 6",Gadesco
4,"Pescarolo ed Uniti, Giuseppe Mazzini, 73","Paderno Ponchielli, Municipio, 3","Pescarolo ed Uniti, Giuseppe Mazzini, 73"
5,"Vescovato, Roma, 44","San Bassano, Mons. Frosi","Cingia de' Botti, Giuseppina, 79"
6,Gadesco,"Fiesco, Giuseppe Parini, 10","Martignana di Po, Libertà, 64"
7,"Piadena, Platina, 40","Soncino, Dante Alighieri, 23","Solarolo Rainerio, Giuseppina, 81"
8,"Solarolo Rainerio, Giuseppina, 81","San Giovanni in Croce, Feudatari, 25 A","San Giovanni in Croce, Feudatari, 25 A"
9,"Gussola, Comaschi, 1",NaN,NaN


In [12]:
#COMPUTE KM LOCAL SEARCH
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur1d[day].dropna(), 1, dfd)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('Ostiano, Mazzini, 1')
    to_write[day] = pd.Series(sch)
to_write.to_csv("../Dati/turni_f/ls1_km_shifts.csv", index=False, sep=";")

In [13]:
week= ['martedi', 'mercoledi', 'giovedi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur2d[day].dropna(), 2, dfd)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('San Giovanni in Croce, Feudatari, 25 A')
    to_write[day] = pd.Series(sch)
to_write.to_csv("../Dati/turni_f/ls2_km_shifts.csv", index=False, sep=";")